In [135]:
import pandas as pd
import os 
import glob
import re
from datetime import datetime, timedelta
import shutil
from dateutil import parser

# Step 1: Initialize the combined dataframe
combined_file = 'combined_reservation_data.csv'
reservation_folder = 'Reservations/'

# Check if the combined_reservation_data.csv file exists
if os.path.exists(combined_file):
    combined_df = pd.read_csv(combined_file)
    print(f"Loaded existing combined data from {combined_file}")
else:
    # If the file does not exist, initialize an empty dataframe
    combined_df = pd.DataFrame()
    print(f"No existing combined file found. Starting with an empty dataframe.")

# Step 2: Read all the new reservation files and append to combined_df
dataframes = []
files = glob.glob('reservation_data_*.csv')

for file in files:
    df = pd.read_csv(file)
    dataframes.append(df)

if dataframes:
    # Combine new dataframes
    new_combined_df = pd.concat(dataframes, ignore_index=True)

    # Append the new data to the existing combined dataframe
    combined_df = pd.concat([combined_df, new_combined_df], ignore_index=True)

    # Step 3: Save the updated combined dataframe
    combined_df.to_csv(combined_file, index=False)
    print(f"Updated combined data saved to {combined_file}")

    # Step 4: Move the processed reservation files to the reservation folder
    if not os.path.exists(reservation_folder):
        os.makedirs(reservation_folder)

    for file in files:
        destination = os.path.join(reservation_folder, os.path.basename(file))
        shutil.move(file, destination)
        print(f"Moved {file} to {destination}")
else:
    print("No new reservation files found to append.")
   

# Step 1: Clean the 'Date' column by splitting at the hyphen and keeping the first part
combined_df['Date'] = combined_df['Date'].str.split(' - ').str[0].str.strip()
# Step 2: Clean the 'Time', 'Fees', and 'Info' columns
# If 'Time' contains a comma, move the value from 'Fees' to 'Time', and move the value from 'Info' to 'Fees'
combined_df['Time'] = combined_df['Time'].astype(str)
for idx, row in combined_df.iterrows():
    if ',' in row['Time']:
        # Move the fee to the time column
        combined_df.at[idx, 'Time'] = row['Fees']
        # Move the info to the fees column
        combined_df.at[idx, 'Fees'] = row['Info']
        
# Displaying cleaned DataFrame as an example
# combined_df

Loaded existing combined data from combined_reservation_data.csv
No new reservation files found to append.


In [136]:
# Step 1: Clean the 'Date' column to remove unnecessary suffixes
def clean_date(date_str):
    # Remove the weekday part (e.g., "Mon, ") if present
    date_str = re.sub(r'^\w{3},\s*', '', date_str)
    # Remove ordinal suffixes like "th", "st", "rd", "nd"
    date_str = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)
    return date_str

def standardize_date(date_str):
    try:
        # Replace common ordinal suffixes (st, nd, rd, th)
        date_str = date_str.replace("st", "").replace("nd", "").replace("rd", "").replace("th", "")
        # Parse the date using dateutil.parser
        parsed_date = parser.parse(date_str)
        # Return the standardized date in YYYY-MM-DD format
        return parsed_date.strftime("%Y-%m-%d")
    except Exception as e:
        return None  # Handle any parsing errors
# Apply the cleaning function to the 'Date' column
combined_df['Date'] = combined_df['Date'].apply(clean_date)
combined_df['Date'] = combined_df['Date'].apply(standardize_date)
# Step 2: Convert the cleaned date strings to actual datetime objects
combined_df['AsOfDate'] = pd.to_datetime(combined_df['Date'], format='%b %d', errors='coerce')

def convert_short_time(time_str):
    # Replace short formats like 9a -> 9:00 AM and 9p -> 9:00 PM
    converted_time_str = re.sub(r'(\d+)([apAP])', r'\1:00 \2M', time_str.upper())
    return converted_time_str

def parse_time(time_str, asof_date):
    try:
        # Remove spaces around the dash
        time_str = re.sub(r'\s*-\s*', '-', time_str)
        
        # Extract start and end time parts
        time_parts = time_str.split('-')
        
        # Define a list of formats to try for start and end times
        time_formats = ['%I%p', '%I:%M %p', '%I%M %p', '%I']
        
        # Try to parse the start and end times with multiple formats
        start_time = None
        end_time = None
        
        for fmt in time_formats:
            try:
                start_time = datetime.strptime(time_parts[0], fmt).time()
                end_time = datetime.strptime(time_parts[1], fmt).time()
                break
            except ValueError:
                continue
        
        # If parsing fails, log the failure
        if start_time is None or end_time is None:
            print(f"Unable to parse: {time_str}")
            return None, None

        # Combine the start time with the date
        start_datetime = datetime.combine(asof_date, start_time)
        
        # Check if the end time is on the next day (i.e., crosses midnight)
        if end_time < start_time:
            end_datetime = datetime.combine(asof_date + timedelta(days=1), end_time)
        else:
            end_datetime = datetime.combine(asof_date, end_time)
        
        return start_datetime, end_datetime
    
    except Exception as e:
        print(f"Error occurred while parsing '{time_str}': {e}")
        return None, None
def extract_fee(fee_str):
    # This regex matches everything up to the first '+'
    fee = re.sub(r'\+.*', '', fee_str).strip()
    fee = re.sub(r'/.*', '', fee).strip()
    return fee


combined_df['Time'] = combined_df['Time'].apply(convert_short_time)
# Apply parsing function and create the columns
combined_df[['StartTime', 'EndTime']] = combined_df.apply(lambda row: pd.Series(parse_time(row['Time'], row['AsOfDate'])), axis=1)
# Calculate the duration (in hours)
combined_df['Duration'] = (combined_df['EndTime'] - combined_df['StartTime']).dt.total_seconds() / 3600
combined_df['Fees'] = combined_df['Fees'].apply(extract_fee)


combined_df.to_csv('combined_reservation_data.csv', index=False)   
print('Done!')


Done!


In [137]:
from datetime import datetime, timedelta

def generate_unlisted_reservations(data):
    # Define the start and end of the day for the courts
    opening_time = datetime.strptime("07:00 AM", "%I:%M %p").time()
    closing_time = datetime.strptime("12:00 AM", "%I:%M %p").time()

    # Extract the court names
    courts = [f"Court #{i}" for i in range(1, 9)]

    # Helper function to create a list of time slots (30-minute increments) for the entire day
    def generate_time_slots(opening_time, closing_time):
        slots = []
        current_time = datetime.combine(datetime.today(), opening_time)
        end_time = datetime.combine(datetime.today(), closing_time) + timedelta(days=1)  # For midnight
        while current_time < end_time:
            slots.append(current_time.time())
            current_time += timedelta(minutes=30)
        return slots

    # Generate time slots for the entire day
    all_time_slots = generate_time_slots(opening_time, closing_time)

    # Initialize a dictionary to track bookings for each court
    court_bookings = {court: {} for court in courts}

    # Iterate through the reservation data and mark the booked time slots by date
    for index, row in data.iterrows():
        court_list = row['Court'].split(', ')
        date = row['Date']
        start_time = datetime.strptime(row['StartTime'].split()[1], "%H:%M:%S").time()
        end_time = datetime.strptime(row['EndTime'].split()[1], "%H:%M:%S").time()

        # Mark time slots as reserved for the specific courts and date
        current_time = datetime.combine(datetime.today(), start_time)
        end_datetime = datetime.combine(datetime.today(), end_time)
        while current_time.time() < end_time:
            for court in court_list:
                if court in court_bookings:
                    if date not in court_bookings[court]:
                        court_bookings[court][date] = {slot: False for slot in all_time_slots}
                    court_bookings[court][date][current_time.time()] = True
            current_time += timedelta(minutes=30)

    # Prepare a dictionary of unlisted reservations
    unlisted_reservations = {}

    for court, dates in court_bookings.items():
        unlisted_reservations[court] = {}
        for date, bookings in dates.items():
            unlisted_times = [time for time, booked in bookings.items() if booked]
            unlisted_reservations[court][date] = unlisted_times

    return unlisted_reservations


import pandas as pd
from datetime import datetime, timedelta

# Load the dataset
file_path = 'combined_reservation_data.csv'
data = pd.read_csv(file_path)

# Define the start and end of the day for the courts
opening_time = datetime.strptime("07:00 AM", "%I:%M %p").time()
closing_time = datetime.strptime("12:00 AM", "%I:%M %p").time()

# Function to generate time slots for the entire day
def generate_time_slots(opening_time, closing_time):
    slots = []
    current_time = datetime.combine(datetime.today(), opening_time)
    end_time = datetime.combine(datetime.today(), closing_time) + timedelta(days=1)  # For midnight
    while current_time < end_time:
        slots.append(current_time.time())
        current_time += timedelta(minutes=30)
    return slots

# Function to generate unlisted reservations per court and date
def generate_unlisted_reservations(data):
    # Extract the court names
    courts = [f"Court #{i}" for i in range(1, 9)]

    # Generate time slots for the entire day
    all_time_slots = generate_time_slots(opening_time, closing_time)

    # Initialize a dictionary to track bookings for each court
    court_bookings = {court: {} for court in courts}

    # Iterate through the reservation data and mark the booked time slots by date
    for index, row in data.iterrows():
        court_list = row['Court'].split(', ')
        date = row['Date']
        start_time = datetime.strptime(row['StartTime'].split()[1], "%H:%M:%S").time()
        end_time = datetime.strptime(row['EndTime'].split()[1], "%H:%M:%S").time()

        # Mark time slots as reserved for the specific courts and date
        current_time = datetime.combine(datetime.today(), start_time)
        while current_time.time() < end_time:
            for court in court_list:
                if court in court_bookings:
                    if date not in court_bookings[court]:
                        court_bookings[court][date] = {slot: False for slot in all_time_slots}
                    court_bookings[court][date][current_time.time()] = True
            current_time += timedelta(minutes=30)

    # Prepare a dictionary of unlisted reservations
    unlisted_reservations = {}

    for court, dates in court_bookings.items():
        unlisted_reservations[court] = {}
        for date, bookings in dates.items():
            unlisted_times = [time for time, booked in bookings.items() if booked]
            unlisted_reservations[court][date] = unlisted_times

    return unlisted_reservations, all_time_slots

# Function to calculate the reservation summary (sum of reserved and free time per court by date)
def calculate_reservation_summary(reservations, all_time_slots):
    summary = {}

    for court, dates in reservations.items():
        summary[court] = {}
        for date, reserved_times in dates.items():
            total_reserved_slots = len(reserved_times)
            total_free_slots = len(all_time_slots) - total_reserved_slots
            summary[court][date] = {
                "Event Slots": total_reserved_slots,
                "Reserved Slots": total_free_slots
            }

    return summary

# Generate unlisted reservations and the summary
unlisted_reservations_with_dates, time_slots = generate_unlisted_reservations(data)
reservation_summary = calculate_reservation_summary(unlisted_reservations_with_dates, time_slots)

# Convert to DataFrame for display
reservation_summary_df = pd.DataFrame.from_dict({(court, date): values
                                                 for court, date_values in reservation_summary.items()
                                                 for date, values in date_values.items()},
                                                orient='index')



In [138]:
def add_reserved_entries(data, reservation_summary):
    # Create a list to hold the new rows
    new_rows = []

    # Iterate through the reservation summary
    for court, date_values in reservation_summary.items():
        # court is the key (e.g., "Court #1"), and date_values is a dictionary of dates and slot info
        for date, values in date_values.items():
            # Check if there are any reserved slots
            if values['Reserved Slots'] > 0:
                # Create a new row with the specified fields
                new_row = {
                    'Title': 'Reserved',
                    'Date': date,
                    'Time': '',
                    'Fees': '$20',
                    'Info': 'Fees For 30 mins slot',
                    'Coach': '',
                    'Court': court,
                    'AdditionalInfo': '',
                    'Registrants': values['Reserved Slots'],
                    'StartTime': '',
                    'EndTime': '',
                    'Duration': values['Reserved Slots']
                }
                # Append the new row to the list
                new_rows.append(new_row)

    # Convert the new rows to a DataFrame
    new_rows_df = pd.DataFrame(new_rows)

    # Concatenate the original data with the new rows
    updated_data = pd.concat([data, new_rows_df], ignore_index=True)

    return updated_data


data = add_reserved_entries(data, reservation_summary)
data.drop(columns=["StartTime","EndTime", "Duration","AsOfDate", "AdditionalInfo"], inplace=True)
data['Fees'] = data['Fees'].replace('[\$,]', '', regex=True).astype(float)
data['Revenue'] = data['Fees'] * data['Registrants']
data.to_csv('combined_total_data.csv', index=False)   


In [139]:
data.dtypes

Title           object
Date            object
Time            object
Fees           float64
Info            object
Coach           object
Court           object
Registrants      int64
Revenue        float64
dtype: object

In [141]:
daily_revenue = data.groupby('Date')['Revenue'].sum().reset_index()

In [142]:
daily_revenue

,Date,Revenue
0,2024-08-17,2600.0
1,2024-08-19,8035.0
2,2024-08-20,8415.0
3,2024-08-21,8735.0
4,2024-08-22,7670.0
5,2024-08-23,6925.0
6,2024-08-24,6740.0
